In [1]:
"""Code assumes the ability to train using a GPU with CUDA.
"""
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR
from advertorch.attacks import GradientSignAttack, CarliniWagnerL2Attack, PGDAttack
import matplotlib.pyplot as plt
from lenet import LeNet

# makes default tensor a CUDA tensor so GPU can be used
torch.set_default_tensor_type('torch.cuda.FloatTensor')
device = 2
torch.cuda.set_device(device)

### Define data loaders and data preprocessing steps

In [2]:
tr_batch_size = 64
te_batch_size = 50

data_preprocess = torchvision.transforms.Compose([
                        torchvision.transforms.ToTensor(),
                        torchvision.transforms.Normalize((0.1307,), (0.3081,))])
# the mean of mnist pixel data is .1307 and the stddev is .3081

train_loader = torch.utils.data.DataLoader(
                    torchvision.datasets.MNIST('./data', train=True, download=True,
                         transform=data_preprocess), 
                    batch_size=tr_batch_size, 
                    shuffle=True)

test_loader = torch.utils.data.DataLoader(
                    torchvision.datasets.MNIST('./data', train=False, download=True,
                         transform=data_preprocess), 
                    batch_size=te_batch_size, 
                    shuffle=False)

### Define adversarial example generation function

In [3]:
def generate_adversarial_samples(og_samples, true_labels, adversary, num_per_samp=1):
    """Create num_per_samp adversarial examples for each sample in
    og_samples. Return the generated samples along with corresponding 
    adv_labels, a tensor containing the adversarial examples' labels.
    """
    adv_samples = []
    for i in range(num_per_samp):
        adv_samples.append(adversary.perturb(og_samples, true_labels))
    adv_samples = torch.cat(adv_samples, 0)
    adv_labels = torch.cat([true_labels]*num_per_samp, 0)
    return adv_samples, adv_labels

### Functions for Jacobian regularization

In [4]:
def bp_matrix(batch_size, n_outputs):
    """Creates matrix that is used to calculate Jacobian for multiple input 
    samples at once.
    """
    idx = torch.arange(n_outputs).reshape(n_outputs,1).repeat(1,batch_size).reshape(batch_size*n_outputs,)
    return torch.zeros(len(idx), n_outputs).scatter_(1, idx.unsqueeze(1), 1.)

In [5]:
def avg_norm_jacobian(x, n_outputs, bp_mat):
    """Returns squared frobenius norm of the input-output Jacobian averaged 
    over the entire batch of inputs in x.
    """
    batch_size = x.shape[0]
    # needed because some edge-case batches are not standard size
    if bp_mat.shape[0]/n_outputs != batch_size:     
        bp_mat = bp_matrix(batch_size, n_outputs)
    x = x.repeat(n_outputs, 1, 1, 1)
    x = Variable(x, requires_grad=True)
    # needed so that we can get gradient of output w.r.t input
    y = net(x)
    x_grad = torch.autograd.grad(y, x, grad_outputs=bp_mat, create_graph=True)[0]
    # get sum of squared values of the gradient values 
    j = x_grad.pow(2).sum() / batch_size
    return j

### Define loss function


In [6]:
def my_loss(output, labels, alpha_wd=0, alpha_jr=0, x=None, bp_mat=None):
    """Adds terms for L2-regularization and the norm of the input-output 
    Jacobian to the standard cross-entropy loss function. Check https://arxiv.org/abs/1908.02729
    for alpha_wd, alpha_jr suggestions.
    """
    # standard cross-entropy loss base
    loss = F.cross_entropy(output, labels)
    
    # add l2 regularization to loss 
    if alpha_wd != 0:
        l2 = 0
        for p in net.parameters():
            l2 += p.pow(2).sum()
        loss = loss + alpha_wd * l2
    
    # add input-output jacobian regularization formulation
    if alpha_jr != 0:
        n_outputs = output.shape[1]
        j = avg_norm_jacobian(x, n_outputs, bp_mat)
        loss = loss + (alpha_jr / 2) * j
        # needed so gradients don't accumulate in leaf variables when calling loss.backward in train function
        optimizer.zero_grad()

    return loss

### Define train and test functions 

In [7]:
def train(alpha_wd, alpha_jr, adversary=None):
    net.train()
    
    for batch_idx, (samples, labels) in enumerate(train_loader):
        # sends to GPU, i.e. essentially converts from torch.FloatTensor to torch.cuda.FloatTensor
        samples, labels = samples.to(device), labels.to(device)
        
        # expand dataset with adversarial examples if adversary specified
        if adversary != None:
            adv_samples, adv_labels = generate_adversarial_samples(samples, labels, adversary)  
            samples, labels = torch.cat([samples, adv_samples], 0), torch.cat([labels, adv_labels], 0)
                
        optimizer.zero_grad()
        
        output = net(samples)
        
        loss = my_loss(output, labels, alpha_wd=alpha_wd, alpha_jr=alpha_jr, x=samples, bp_mat=tr)
        loss.backward()
        
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            j = avg_norm_jacobian(samples, output.shape[1], tr)
            print(f'\tLoss: {loss.item():.6f} Average norm of Jacobian: {j:6f}')
            train_losses.append(loss.item())
            jacobian_norms.append(j)

In [8]:
def test(alpha_wd, alpha_jr):
    net.eval()
    test_loss = 0
    correct = 0
    
    for samples, labels in test_loader:
        samples, labels = samples.to(device), labels.to(device)
        output = net(samples)
        test_loss += my_loss(output, labels, alpha_wd=alpha_wd, alpha_jr=alpha_jr, x=samples, bp_mat=te).item()
        # output is a tensor, .data retrieves its data, max returns the index of the highest valued element
        preds = output.data.max(1, keepdim=True)[1]
        correct += preds.eq(labels.data.view_as(preds)).sum().item()
                
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * float(correct / len(test_loader.dataset))
    
    print(f'\tTest set accuracy: ({test_accuracy:.2f}%)')
    
    test_accuracies.append(test_accuracy)
    test_losses.append(test_loss)

### Training

In [10]:
# training details
n_epochs = 30
log_interval = 200
training_round = 1
torch.manual_seed(training_round)

# varying values for certain hyperparameters to produce models with varying degrees of robustness
epsilons = [.1, .2, .3, .4, .5, .6]
alpha_wds = [0, .0000001, .000001, .00001, .0001, .001, .01]
alpha_jrs = [.01, .000001, .00001, .0001, .001, .01]

# dictionary to record each model's training/testing stats
performance = {}

for hyp_param_to_vary in [alpha_jrs]:
    epsilon = 0
    alpha_wd = 0
    alpha_jr = 0
    
    for value in hyp_param_to_vary:        
        # change hyperparameter that is being varied
        if hyp_param_to_vary == epsilons:
            epsilon = value
            tr = bp_matrix(tr_batch_size*2, 10)
            te = bp_matrix(te_batch_size*2, 10)
        elif hyp_param_to_vary == alpha_wds:
            alpha_wd = value
            tr = bp_matrix(tr_batch_size, 10)
            te = bp_matrix(te_batch_size, 10)
        else:
            alpha_jr = value
            tr = bp_matrix(tr_batch_size, 10)
            te = bp_matrix(te_batch_size, 10)
        
        print(f'\nBeginning training for model: models/training_round_{training_round}/ep{epsilon}_wd{alpha_wd}_jr{alpha_jr}_{training_round}')

        # instantiate model and optimizer
        learning_rate = 0.01
        momentum = 0.9
        net = LeNet()
        optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)
        lr_decayer = StepLR(optimizer, step_size=10, gamma=0.1)

        # make model CUDA enabled and define GPU/device to use
        net.cuda()
        
        # define adversary to train against if needed
        FGSM = None
        if epsilon != 0:
            FGSM = GradientSignAttack(predict=net, loss_fn=F.cross_entropy, 
                            eps=epsilon, clip_min=-3., clip_max=3., targeted=False)

        # for tracking training progress
        train_losses = []
        test_losses = []
        test_accuracies = []
        jacobian_norms = []

        for epoch in range(1, n_epochs + 1):
            print(f'Epoch #{epoch}')
            train(alpha_wd=alpha_wd, alpha_jr=alpha_jr, adversary=FGSM)
            test(alpha_wd=alpha_wd, alpha_jr=alpha_jr)
            lr_decayer.step()
        
        performance[f'ep{epsilon}_wd{alpha_wd}_jr{alpha_jr}_{training_round}'] = (train_losses, test_losses, test_accuracies, jacobian_norms)
        torch.save(net.state_dict(), f'models/training_round_{training_round}/ep{epsilon}_wd{alpha_wd}_jr{alpha_jr}_{training_round}')
        


Beginning training for model: models/training_round_1/ep0_wd0_jr0.01_1
Epoch #1
	Loss: 2.324397 Average norm of Jacobian: 0.021185
	Loss: 0.453366 Average norm of Jacobian: 6.163872
	Loss: 0.616027 Average norm of Jacobian: 10.719622
	Loss: 0.219482 Average norm of Jacobian: 11.795981
	Loss: 0.405770 Average norm of Jacobian: 9.498756
	Test set accuracy: (97.32%)
Epoch #2
	Loss: 0.227478 Average norm of Jacobian: 8.986900
	Loss: 0.279715 Average norm of Jacobian: 10.890207
	Loss: 0.184236 Average norm of Jacobian: 10.843582
	Loss: 0.201376 Average norm of Jacobian: 8.624828
	Loss: 0.193151 Average norm of Jacobian: 8.790615
	Test set accuracy: (97.92%)
Epoch #3
	Loss: 0.228385 Average norm of Jacobian: 9.414557
	Loss: 0.120254 Average norm of Jacobian: 10.189798
	Loss: 0.122917 Average norm of Jacobian: 6.988352
	Loss: 0.275382 Average norm of Jacobian: 7.613367
	Loss: 0.145000 Average norm of Jacobian: 8.814926
	Test set accuracy: (98.53%)
Epoch #4
	Loss: 0.102872 Average norm of Jac

KeyboardInterrupt: 

### Write performance dictionary to text file

In [ ]:
f = open(f'models/training_round_{training_round}/training_round_{training_round}_performance.txt','w')
f.write(str(performance))
f.close()

# to read dictionary from file:
# f = open(f'models/training_round_{training_round}_performance.txt','r')
# d = eval(f.read())